### Метрики эффективности моделей машинного обучения

#### Цель работы

Научиться измерять эффективность моделей машинного обучения с помощью метрик, вибирать метрики исходя из задачи, разбивать датасет на обучающую и тестовую подвыборки.

#### Содержание работы

1. Загрузите данные о вероятности развития сердечного приступа, прилагающийся к этой работе (heart.csv).
1. Обучите на этих данных простую модель логистической регрессии и выведите метрику точности (accuracy).
1. Разделите датасет на две части - первую половину используйте для обучения, а вторую - для оценки точности. Сравните значения метрик.
1. Разделите датасет на две части случайным образом. Повторите анализ.
1. Разделите датасет с помощью библиотечной функции. Повторите анализ несколько раз.
1. Постройте матрицу классификации и отчет о классификации для обученной модели для обучающей и тестовой выборок. Проинтерпретируйте полученные значения.
1. Подсчитайте для построенной модели значение всех метрик эффективности классификации на тестовой и обучающей выборках. Нужно использовать следующие метрики: accuracy, precision, recall, f1.

#### Методические указания

In [52]:
import pandas as pd
import numpy as np

data = pd.read_csv(r'heart.csv')
data

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


После чтения файла вы должны увидеть примерно такую таблицу при выводе первых строк датасета:

|index|age|sex|cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|0|63|1|3|145|233|1|0|150|0|2\.3|0|0|1|1|
|1|37|1|2|130|250|0|1|187|0|3\.5|0|0|2|1|
|2|41|0|1|130|204|0|0|172|0|1\.4|2|0|2|1|
|3|56|1|1|120|236|0|1|178|0|0\.8|2|0|2|1|
|4|57|0|0|120|354|0|1|163|1|0\.6|2|0|2|1|

Как всегда выделим целевую переменную, сразу же импортируем и обучим модель логистической регрессии:

In [53]:
y = data["output"]
x = data.drop("output", axis=1)

from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression().fit(x, y)
logistic.score(x, y)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8547854785478548

Данная модель показывает более 85% точности. Но эта оценка качества модели является завышенной (оптимистичной), так как она оценена именно по тем данным, на которых модель училась. Чтобы оценить именно обобщающую способность модели, то есть ее способность правильно предсказывать значение целевой переменной для тех объектов, которые она не учитывала при подстройке параметров, необходимо оценить модель на другой выборке. Для этого нужно разделить исходный датасет на две части. Обучающая выборка (первая часть) будет использоваться для нахождения оптимальных значений внутренних параметров модели, а тестовая выборка (вторая часть) - для оценки качества полученной модели.

##### Разделение выборки

Делить датасет можно разными способами. Важно, чтобы каждый конкретный объект попал только в одну выборку - либо тестовую, либо обучающую. То есть эти части должны быть непересекающиеся. Самый простой способ - просто взять какое-то количество объектов в начале датафрейма в обучающую выборку, а остальные - в тестовую. То есть мы просто берем несколько первых строчек (например, 200) для обучающей выборки:

In [54]:
x_train, y_train = x[:200], y[:200]

Важно убедиться, что все в порядке с формами получившихся массивов:

In [55]:
x_train.shape, y_train.shape

((200, 13), (200,))

Итак, первые 200 строк попали в обучающую выборку. Тогда тестовую выборку составять оставшиеся строки датасета:

In [56]:
x_test, y_test = x[200:], y[200:]
x_test.shape, y_test.shape

((103, 13), (103,))

Получилось, что в тестовой выборке осталось 103 объекта. Теперь можно заново обучить модель классификации и оценить ее качество. Обратите внимание, что мы вызываем метод _fit()_ именно на обучающей части датасета. А вот эффективность измеряем сначала на обучающей части, а затем на тестовой, чтобы сравнить:

In [57]:
logistic_test = LogisticRegression().fit(x_train, y_train)
logistic_test.score(x_train, y_train), logistic_test.score(x_test, y_test)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.9, 0.5436893203883495)

Получилось, что на обучающей выборки точность модели даже немного повысилась, до 90%. Это произошло за счет того, что в обучающей выборке меньше данных, чем в целом датасете. А чем меньше точек, тем проще модели под них подстроиться. А вот эффективность модели на тестовых данных стала сильно ниже - всего 54%. Это значит, что наша модель чуть лучше, чем простое угадывание. Хотя, как увидим дальше, на самом деле не все так плохо.

Вместо того, чтобы вручную задавать количество объектов, которые отводятся на обучающую выборку, можно выразить это количество через процент от всего объема датасета. Это даст нам универсальность:

In [58]:
N = int(x.shape[0] * 0.8)

x_train, y_train, x_test, y_test = x[:N], y[:N], x[N:], y[N:]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((242, 13), (242,), (61, 13), (61,))

В данном примере мы взяли 80% объектов для обучающей выборки и 20% - оставили на тестовую. Это довольно стандартная схема разбиения и мы в общем случае будем придерживаться именно ее. Вот какие формы получились у массивов:

In [59]:
((242, 13), (242,), (61, 13), (61,))

((242, 13), (242,), (61, 13), (61,))

Давайте еще раз обучим и оценим модель, уже на новом разделении:

In [60]:
logistic_test = LogisticRegression().fit(x_train, y_train)
logistic_test.score(x_train, y_train), logistic_test.score(x_test, y_test)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8884297520661157, 0.6229508196721312)

Мы видим, что точность на обучающей выборке (мы ее будем называть обучающей точностью) чуть опустилась - опять же это эффект количества точек - сейчас их чуть больше. Но и тестовая эффективность (точность модели, измеренная на тестовой выборке) тоже подросла - до 62%. Это потому, что чем больше примеров мы использовали для обучения, тем более качественной и обобщающей получается наша модель.

Но разнича между обучающей и тестовой точностью у нашей модели все еще довольно высока. С чем это может быть связано? На самом деле, мы поступаем немного неправильно. Нам нужно более внимательно присмотреться к данным. Мы уже видели первые несколько строк датасета. Давайте еще выведем последние несколько строк:

In [61]:
data.tail()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0
302,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0


|index|age|sex|cp|trtbps|chol|fbs|restecg|thalachh|exng|oldpeak|slp|caa|thall|output|
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
|298|57|0|0|140|241|0|1|123|1|0\.2|1|0|3|0|
|299|45|1|3|110|264|0|1|132|0|1\.2|1|0|3|0|
|300|68|1|0|144|193|1|1|141|0|3\.4|1|2|3|0|
|301|57|1|0|130|131|0|1|115|1|1\.2|1|1|3|0|
|302|57|0|1|130|236|0|0|174|0|0\.0|1|1|2|0|

Можно заметить, что вы первой половине данных целевая переменная всегда положительная, а во второй - отрицательная. Другими словами, датасет отсортирован по значению целевой переменной. Поэтому наше разбиение имеет один неприятный эффект - в обучающей выборке скапливается большое количество положительных примеров, а в тестовой - только отрицательные. Другими словами, обучающая и тестовая выборки очень непохожи друг надруга. А для того, чтобы наше оценивание сработало, нужно, чтобы они были как можно более однородными.

Поэтому просто выбрать первые строки датасета обычно не очень рациональное решение. Даже если датасет не отсортирован по значению целевой переменной, он вполне может быть упорядочен по какому-то другому признаку, или хотя бы по времени добавления объектов в набор или базу данных. А такое упорядочение вносит систематическую ошибку выборки, которая смещает оценку качества модели.

Из-за этого делить данные на обучающую и тестовую выборки практически всегда нужно случайным образом. То есть выбрать случайное подмножество точек и поместить их в обучающий набор, а оставшиеся - в тестовый. Основная техническая трудность здесь состоит в том, что датасет у нас разделен на две переменные - матрицу признаков и вектор значений целевой переменной. Просто воспользоваться методом случайного выбора из массива мы не можем, так как когда мы повторим его два раза - для переменных _x_ и _y_ у нас будут выбраны не соответствующие друг другу объекты.

Для решения этой проблемы воспользуемся индексными масками. Заведем массив булевских значений, в которых количество истинных значений будет соответствовать желаемому объему обучающей выборки, а ложных - объему тестовой:

In [62]:
mask = np.array([True] * N + [False] * (y.shape[0] - N))

В сумме должно получиться в точности количество объектов в полном датасете.Теперь перемешаем этот массив, используя стандартную функцию _numpy_:

In [63]:
from numpy.random import shuffle

shuffle(mask)
mask

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True, False,  True, False,  True,
        True,  True,  True,  True,  True, False,  True, False,  True,
       False,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True, False,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True, False,  True, False,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True, False,  True,  True, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True, False,
        True,  True,

Этот булев массив можно использовать как маску при индексировании исходного датасета. Если выбрать данные по этой маске, то в итог попадут только те элементы, которые стоят на тех местах, на которых в маске - истина:

In [64]:
x_train = x[mask]
x_train.shape

(242, 13)

Смысл этого приема в том, что мы можем применить одну и ту же маску к обоим частям массива. А если инвертировать эту маску, то выберем оставшиеся значения:

In [65]:
x_train, y_train, x_test, y_test = x[mask], y[mask], x[~mask], y[~mask]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((242, 13), (242,), (61, 13), (61,))

После такого разделения опять обучим и оценим модель:

In [66]:
logistic_test = LogisticRegression().fit(x_train, y_train)
logistic_test.score(x_train, y_train), logistic_test.score(x_test, y_test)

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.8388429752066116, 0.8524590163934426)

Мы видим, что теперь разница между тестовой и обучающей выборкой стала гораздо меньше - 88% и 77% соответственно. Это как раз обосновано более правильным разделением, которое дает однородные по своему составу части датасета.

Конечно, такая стандартная процедура как случайное разделение датасета на две части уже реализована в виде готовой функции - в реальной работе не придется каждый раз дублировать приведенный код. Такая функция лежит в пакете _sklearn_, который отвечает за методологию выбора модели - обэтом мы поговорим в следующих работах:

In [67]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((242, 13), (242,), (61, 13), (61,))

Обратите внимание, что библиотечная функция возвращает элементы в строго определенном порядке. Его придется либо запомнить, либо пользоваться готовыми сниппетами кода.

##### Построение метрик качества классификации

До сих пор мы оценивали модель только по одной метрике эффективности. Но для более полного анализе этого недостаточно. В пакете _metrics_ собрано множество функций, которые позволяют исследовать поведение уже обученных моделей, в том числе - метрики эффективности. Импортируем несколько нужных нам инструментов:

In [68]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

Для использования большинства метрик необходимо передавать в эти функции два вектора - вектор истинных значений целевой переменной и вектор предсказанных значений. Истинные (эмпирические) значения - это часть исходного датасета. А предсказанные (теоретические) значения можно вычислить. Рассчитаем предсказанные значения отдельно для обучающей и для тестовой выборки:

In [69]:
y_test_pred = logistic_test.predict(x_test)
y_train_pred = logistic_test.predict(x_train)

Первым делом построим матрицу классификации::

In [70]:
confusion_matrix(y_train, y_train_pred)

array([[ 81,  28],
       [ 14, 119]], dtype=int64)

Она показывает, как часто и в каких классах модель ошибается. В идеальной модели ненулевые значения должны быть только на главной диагонали:

Обратите внимание, что для правильного анализа модели нужно строить все эти диагностические инструменты именно по тестовой выборке:

In [71]:
confusion_matrix(y_test, y_test_pred)

array([[25,  4],
       [ 2, 30]], dtype=int64)

Именно эти данные покажут истинное поведение модели.

Еще один полезный инструмент - отчет о классификации:

In [72]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.93      0.86      0.89        29
           1       0.88      0.94      0.91        32

    accuracy                           0.90        61
   macro avg       0.90      0.90      0.90        61
weighted avg       0.90      0.90      0.90        61



Он показывает всю основную информацию по итогам классификации. Познакомьтесь со структурой данного отчета и проинтерпретируйте полученные результаты.

Что касается остальных метрик качества классификации, можно строить их отдельно:

In [73]:
precision_score(y_test, y_test_pred)

0.8823529411764706

Но более удобно свести их все в таблицу при помощи датафрейма:

In [74]:
metrics = pd.DataFrame({
    "Train": [
        accuracy_score(y_train, y_train_pred),
        precision_score(y_train, y_train_pred),
        recall_score(y_train, y_train_pred),
        f1_score(y_train, y_train_pred),
    ],
    "Test": [
        accuracy_score(y_test, y_test_pred),
        precision_score(y_test, y_test_pred),
        recall_score(y_test, y_test_pred),
        f1_score(y_test, y_test_pred),
    ],
}, index = ["Accuracy", "Precision", "Recall", "F1"])

metrics

,Train,Test
Accuracy,0.826446,0.901639
Precision,0.809524,0.882353
Recall,0.894737,0.937500
F1,0.850000,0.909091


Здесь мы видим сравнение четырех разных основных метрик качества модели классификации по обучающей и по тестовой выборкам. Видно, что тестовые метрики все чуть ниже, чем обучающие. Это доволно типичное поведение моделей машинного обучения. Мы уже рассматривали причины этого. Но помните, что это не гарантируется и в конкретном случае, тестовые метрики могут быть как очень близки, так и даже выше. Все решает случай при рандомном разеделении выборки.

При оценке эффективности модели машинного обучения ориентирвоаться нужно только на тестовые метрики. Но сравнение уровня тестовой и обучающей метрики - это один их основных инструментов диагностики модели машинного обучения.

#### Задания для самостоятельного выполнения

1. Повторите анализ для других видов моделей. Используйте 5-10 разных классов моделей. Подсчитывайте только метрики на тестовой выборке.
1. Повторите анализ для другого датасета по вашему выбору. Используйте несколько моделей для сравнения. Используйте датасет для множественной классификации.
1. Повторите анализ для датасета, предназначенного для решения задачи регрессии. Используйте все метрики качества регрессии, изученные на лекции. Постройте 5 - 10 разных моделей регрессии.

##### 1

In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import time

# Загрузите данные (убедитесь, что файл heart.csv доступен)
data = pd.read_csv('heart.csv')

# Определим признаки и целевую переменную
X = data.drop('output', axis=1)
y = data['output']

# Разделение на обучающую (80%) и тестовую (20%) выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Словарь моделей для сравнения
models = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVC': SVC(probability=True),
    'KNeighbors': KNeighborsClassifier(),
    'GaussianNB': GaussianNB()
}


# Словарь для хранения результатов
results = {}
for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1': f1_score(y_test, y_pred),
        'time': time.time() - start
    }
    
# Вывод результатов в виде таблицы
results_df = pd.DataFrame(results).T
results_df

,Accuracy,Precision,Recall,F1,time
LogisticRegression,0.885246,0.878788,0.90625,0.892308,0.136424
DecisionTree,0.803279,0.916667,0.68750,0.785714,0.013189
RandomForest,0.852459,0.848485,0.87500,0.861538,0.249929
SVC,0.704918,0.666667,0.87500,0.756757,0.025133
KNeighbors,0.688525,0.685714,0.75000,0.716418,0.015271
GaussianNB,0.868852,0.900000,0.84375,0.870968,0.009324


##### 2

In [76]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Загрузка набора данных iris
iris = load_iris()
X = iris.data
y = iris.target

# Разбиение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Модели для сравнения
models = {
    'LogisticRegression': LogisticRegression(max_iter=200),
    'DecisionTree': DecisionTreeClassifier(),
    'RandomForest': RandomForestClassifier(),
    'SVC': SVC(probability=True)
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred, average='macro'),
        'Recall': recall_score(y_test, y_pred, average='macro'),
        'F1': f1_score(y_test, y_pred, average='macro'),
        'time': time.time() - start
    }
    
results_df = pd.DataFrame(results).T
results_df

,Accuracy,Precision,Recall,F1,time
LogisticRegression,1.0,1.0,1.0,1.0,0.052712
DecisionTree,1.0,1.0,1.0,1.0,0.056819
RandomForest,1.0,1.0,1.0,1.0,0.181417
SVC,1.0,1.0,1.0,1.0,0.185495


##### 3

In [77]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

# Загрузка набора данных California Housing
housing = fetch_california_housing()
X = housing.data
y = housing.target

# Разбиение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

# Словарь моделей для регрессии
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'DecisionTree': DecisionTreeRegressor(),
    'RandomForest': RandomForestRegressor(),
    #'GradientBoosting': GradientBoostingRegressor(),
    'SVR': SVR(),
    'KNeighbors': KNeighborsRegressor()
}

results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    results[name] = {
        'MSE': mean_squared_error(y_test, y_pred),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred),
        'time': time.time() - start
    }
    
results_df = pd.DataFrame(results).T
results_df

,MSE,MAE,R2,time
LinearRegression,0.555892,0.533200,0.575788,0.225232
Ridge,0.555803,0.533204,0.575855,0.230231
DecisionTree,0.486642,0.450044,0.628633,0.569910
RandomForest,0.258924,0.329131,0.802410,15.144984
SVR,1.332012,0.859951,-0.016485,27.753834
KNeighbors,1.118682,0.812798,0.146310,27.781362


#### Контрольные вопросы

1. Почему для анализа модели нужно применять несколько метрик эффективности?
1. Зачем для анализа качества модели делить датасет на обучающую и тренировочную выборки?
1. В чем особенность и область применения каждой метрики качества?
1. Для каждой метрики классификации и регрессии придумайте пример, в котором данная метрика будет определяющей для выбора наилучшей модели.
1. В каких случаях нельзя делить выборку случайным образом?
1. Зачем нужен и как использовать отчет о классификации

#### Дополнительные задания

1. Постройте график зависимости метрик качества классификации из методических указаний от процента выборки, отводимого на тестовую часть.
1. Создайте универсальную функцию, которая подсчитывает все метрики по любой модели и выводит их в виде таблицы.
1. Создайте функцию, принимающая на вход датасет, которая проверяет наибольшее количество разных моделей машинного обучения и выдающая датафрейм с таблицей всех метрик.